In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from mazegenfromc import generate_maze

Tạo mê cung

In [3]:
def mazetensor(maze_np):
    maze_tensor = torch.from_numpy(maze_np).float()
    
    # Batch_size, Channels, Height, Width
    # Số lượng, số lớp, chiều cao, chiều rộng
    #  1 ảnh, 1 màu, cao 10, rộng 10
    maze_tensor = maze_tensor.unsqueeze(0).unsqueeze(0)
    
    return maze_tensor

Hàm lấy vị trí đích

In [4]:
def get_goal_position(maze):

    result = np.where(maze == 9)
    
    y = int(result[0][0])
    x = int(result[1][0])
    return (y, x)
    

Hàm lấy vị trí agent hiện tại

In [5]:
def get_current_position(maze):
    
    result = np.where(maze == 2)
    
    y = int(result[0][0])
    x = int(result[1][0])
    return (y, x)


Hàm lấy tầm nhìn

In [6]:
def get_9x9_view(maze_np, heatmap_np, agent_position):
    pad_size = 4
    
    # Padding
    padded_maze = np.pad(maze_np, pad_size, mode='constant', constant_values=1)
    padded_heat = np.pad(heatmap_np, pad_size, mode='constant', constant_values=0)
    
    y, x = agent_position[0] + pad_size, agent_position[1] + pad_size
    
    # Cắt vùng 9x9
    maze_cut = padded_maze[y-4:y+5, x-4:x+5]
    heat_cut = padded_heat[y-4:y+5, x-4:x+5]
    
    heat_norm = 1.0 / (1.0 + heat_cut)
    
    # Chồng thành Tensor [2, 9, 9]
    stack_map = torch.stack([
        torch.from_numpy(maze_cut).float(),
        torch.from_numpy(heat_norm).float()
    ], dim=0)
    
    # Thêm chiều [1, 2, 9, 9]
    return stack_map.unsqueeze(0)

Hàm lấy vector chỉ hướng đích

In [7]:
def get_goal_vector(agent_position, goal_position, maze_size):

    #agent_pos: (y, x)
    #goal_pos: (y, x)
    #maze_size: (H, W)

    H, W = maze_size
    
    # 1. Tính khoảng cách thô
    dy = goal_position[0] - agent_position[0]
    dx = goal_position[1] - agent_position[1]
    
    # 2. Chuẩn hóa về khoảng [-1, 1]
    dy_norm = dy / H
    dx_norm = dx / W
    
    return torch.tensor([dy_norm, dx_norm], dtype=torch.float32)

Hàm cập nhật ô đã đi

In [8]:
def update_visit_count(heatmap, agent_position):
    y, x = agent_position
    heatmap[y, x] += 1
    return heatmap

Tạo hàm ánh xạ số trong lớp quyết định và hướng đi của agent

In [9]:
def get_next_position(current_position, action):

    y, x = current_position
    
    action_map = {
        0: (-1, 0), # Lên
        1: (1, 0),  # Xuống
        2: (0, -1), # Trái
        3: (0, 1)   # Phải
    }
    
    dy, dx = action_map[action]
    next_y = y + dy
    next_x = x + dx
    
    return (next_y, next_x)

Tạo các lớp tích chập (convolution layer)

In [10]:
conv_layers = nn.Sequential(
    # Lớp 1: 9*9 -> 7*7
    nn.Conv2d(in_channels=2, out_channels=16, kernel_size=3), 
    nn.ReLU(),

    # Lớp 2: 7*7 -> 5*5
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
    nn.ReLU(),

    # Lớp 3: 5*5 -> 3*3
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
    nn.ReLU(),

    # Lớp 4: 3*3 -> 1*1
    nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3),
    nn.ReLU()
)

Tạo lớp quyết định

In [11]:
decision_layer = nn.Linear(66, 4)

Tạo hàm dự đoán bước tiếp theo

In [12]:
def predict_move(maze_np, heatmap_np, conv_layers, decision_layer, goal_position):

    current_position = get_current_position(maze_np)
    
    live_view = get_9x9_view(maze_np, heatmap_np, current_position)
    view_4d = conv_layers(live_view)
    view_flat = torch.flatten(view_4d, start_dim=1)  # [1, 64, 1, 1] -> [1, 64]
    
    goal_vec = get_goal_vector(current_position, goal_position, maze_np.shape).unsqueeze(0)  # [dy_norm, dx_norm] -> [[dy_norm, dx_norm]]
    
    final_input = torch.cat((view_flat, goal_vec), dim=1)
    
    action_logits = decision_layer(final_input)
    action = torch.argmax(action_logits, dim=1).item()
    
    next_pos = get_next_position(current_position, action)
    
    return action, next_pos

In [ ]:
maze_np = generate_maze(30)
heatmap_np = np.zeros_like(maze_np, dtype=np.float32)
goal_position = get_goal_position(maze_np)